In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import os

In [13]:
! pip install -U torchviz -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [ ]:
# random seed
def seed_torch():
    seed = 1024
    # seed = int(time.time()*256)
    # 保存随机种子
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_torch()



In [7]:
seed_torch()
item_embs = torch.randn(10, 1, 4, 3)
user_embs = torch.randn(10, 3)
w2_embs = nn.Embedding(10, 6)
b2_embs = nn.Embedding(10, 1)
list = [i for i in range(10)]


neg_numpy = np.array(list)
list = torch.from_numpy(neg_numpy)
print(item_embs)
print(w2_embs)
list

tensor([[[[-1.1620,  1.3113,  0.1507],
          [ 2.2698,  1.3304,  1.2262],
          [ 1.0735, -1.1169,  1.4001],
          [ 1.1982, -0.6696,  0.3269]]],


        [[[-1.2610,  1.0990,  0.3787],
          [-0.3478,  0.1554,  1.5355],
          [ 1.4722,  1.6234, -0.7736],
          [ 0.8810, -0.0651, -1.3484]]],


        [[[-0.7855,  0.2748,  1.0272],
          [-0.1696,  1.9930, -0.2164],
          [ 0.9729,  1.1259,  0.9651],
          [-1.3539, -2.8875,  0.4818]]],


        [[[-0.8747,  0.7910, -1.4992],
          [-0.8333, -0.1704, -1.1794],
          [ 0.1153,  0.0394, -0.8305],
          [ 0.3081, -0.6059, -0.1193]]],


        [[[-0.7071, -0.4894,  1.6141],
          [ 0.1807, -1.6498,  1.8776],
          [ 1.7053, -1.2480,  0.1803],
          [-1.0299, -0.7116,  0.9382]]],


        [[[-0.6594,  1.3287, -0.8353],
          [ 0.5301,  0.6000,  0.0884],
          [ 0.3265,  1.8383, -0.2714],
          [ 0.9953,  0.1639,  0.5745]]],


        [[[-0.3917,  1.1877, -0.2866],
 

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=torch.int32)

In [ ]:
# 原始网络 
class caser(nn.Module):
    def __init__(self):
        super(caser, self).__init__()
        # 测试测试
        L = 4
        dims = 3
        self.n_h = 4
        self.n_v = 3
        self.drop_ratio = 0.5
        self.ac_conv = F.relu
        self.ac_fc = F.relu
        # user and item embeddings
        self.user_embeddings = nn.Embedding(1, 3)
        self.item_embeddings = nn.Embedding(4, 3)
        # 竖直卷积 其 n_v为 3 l为4 
        self.conv_v = nn.Conv2d(1, self.n_v, (L, 1))
        # horizontal conv layer 水平卷积  
        lengths = [i + 1 for i in range(L)]
        # n_h 为 4
        self.conv_h = nn.ModuleList([nn.Conv2d(1, self.n_h, (i, dims)) for i in lengths])
        # fully-connected layer
        self.fc1_dim_v = self.n_v * dims
        self.fc1_dim_h = self.n_h * len(lengths)
        fc1_dim_in = self.fc1_dim_v + self.fc1_dim_h
        # W1, b1 can be encoded with nn.Linear
        self.fc1 = nn.Linear(fc1_dim_in, dims)
        # W2, b2 are encoded with nn.Embedding, as we don't need to compute scores for all items
        self.W2 = nn.Embedding(4, dims+dims)
        self.b2 = nn.Embedding(4, 1)

        # dropout
        self.dropout = nn.Dropout(self.drop_ratio)

        # weight initialization
        self.user_embeddings.weight.data.normal_(0, 1.0 / self.user_embeddings.embedding_dim)
        self.item_embeddings.weight.data.normal_(0, 1.0 / self.item_embeddings.embedding_dim)
        self.W2.weight.data.normal_(0, 1.0 / self.W2.embedding_dim)
        self.b2.weight.data.zero_()

        self.cache_x = None

    def forward(self, item_embs, user_emb):
        # Convolutional Layers
        out, out_h, out_v = None, None, None
        # vertical conv layer
        if self.n_v:
            out_v = self.conv_v(item_embs)
            out_v = out_v.view(-1, self.fc1_dim_v)  # prepare for fully connect

        # horizontal conv layer
        out_hs = list()
        if self.n_h:
            for conv in self.conv_h:
                conv_out = self.ac_conv(conv(item_embs).squeeze(3))
                pool_out = F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
                out_hs.append(pool_out)
            out_h = torch.cat(out_hs, 1)  # prepare for fully connect


        # Fully-connected Layers
        out = torch.cat([out_v, out_h], +-1)
        # apply dropout
        out = self.dropout(out)

        # fully-connected layer
        z = self.ac_fc(self.fc1(out))
        x = torch.cat([z, user_emb], 1)

        w2 = self.W2(torch.from_numpy(np.array(4)))
        b2 = self.b2(torch.from_numpy(np.array(4)))

        if 0:
            w2 = w2.squeeze()
            b2 = b2.squeeze()
            res = (x * w2).sum(1) + b2
        else:
            res = torch.baddbmm(b2, w2, x.unsqueeze(2)).squeeze()

        return res
        

In [ ]:
# 实例化网络`b`
net = caser()
res = net(item_embs, user_embs)
res

In [ ]:
# 水平卷积 + 增加最大池化
# 还是得拆散这来
class demo1(nn.Module):
    def __init__(self):
        super(demo1, self).__init__()
        # 测试测试
        L = 4
        dims = 3
        self.n_h = 4
        self.n_v = 3
        self.drop_ratio = 0.5
        self.ac_conv = F.relu
        self.ac_fc = F.relu

        # horizontal conv layer 水平卷积  
        lengths = [i + 1 for i in range(L)]
        # n_h 为 4
        # modulelist 组
        self.conv_h = nn.ModuleList([nn.Conv2d(1, 4, (i, dims)) for i in lengths])

    def forward(self, item_embs):
        out_hs = list()
        if self.n_h:
            for conv in self.conv_h:
                # squeeze 挤压
                # unzip 解压
                conv_out = self.ac_conv(conv(item_embs).squeeze(3))
                # maxpool1d 方法 。。。
                # squeeze(2) 这个操作在这里是 消边
                pool_out = F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
                out_hs.append(pool_out)
            # 输出在一维进行拼接 
            # 意思就是在二维进行拼接
            # 我的脑子突然有冒出个有意思的想法
            # concat这个操作不是拼接嘛
            # 一个一个的对象 凭借成一对一对的（最近的情况哈哈）
            # 既然是拼接 那么带我们对应拼接的维度 就应该有所增加
            # 这里的 tensor 为 4个 [1,4] 的向量
            # concat（0） 的话 就会成为 [4, 4] 
            # concat（1） 的话 就会变成 [1,16]
            out_hs = torch.cat(out_hs, 1)

        return out_hs
        
net =  demo1()
res = net(item_embs)
print(net._modules)
print(res)
print(res.size())

In [ ]:
# 测试测试 乖乖 我写个模块 写了这么久 看来不能不吃饭 写代码 有点蠢 
# ok 我吃饱了
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        self.conv = nn.Conv2d(1, 4, (2, 3))

    def forward(self, item_embs):
        # 因为最大池化这个操作是 1d的 所以这里需要squeeze(3) 来吧2d 变成 1d
        # 其感觉就是 [1,1,4,1] -> [1,1,4]
        conv_out = self.conv(item_embs).squeeze(3)
        #最大池化 消边
        pool_out = F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
        return pool_out

net = test()
res = net(item_embs)
print(res)
print(res.size())
# print(res.squeeze(3))
# print(res.squeeze(3).size())

In [ ]:
# 竖直卷积
class demo2(nn.Module):
    def __init__(self):
        super(demo2, self).__init__()
        # 测试测试
        L = 4
        dims = 3
        self.n_h = 4
        # 定义竖直卷积通道数
        self.n_v = 3
        self.drop_ratio = 0.5
        self.ac_conv = F.relu
        self.ac_fc = F.relu
        self.fc1_dim_v = self.n_v * dims
        self.conv = nn.Conv2d(1, self.n_v, (L, 1))
    
    def forward(self, item_embs):
        out_v = self.conv(item_embs)
        out_v = out_v.view(-1, self.fc1_dim_v)
        return out_v

item_embs = torch.randn(10, 1, 4, 3)
net = demo2()
res = net(item_embs)
print(res)
print(res.size())

In [ ]:
# demo1 + demo2 concat linear bingo! 
class conv_demo(nn.Module):
    def __init__(self):
        super(conv_demo, self).__init__()
        # 先搬一搬 数据
        L = 4
        dims = 3
        self.n_h = 4
        self.n_v = 3
        self.drop_ratio = 0.5
        self.ac_conv = F.relu
        self.ac_fc = F.relu
        # 搬完了
        
        # horizontal conv layer 水平卷积  
        lengths = [i + 1 for i in range(L)]
        # n_h 为 4
        # modulelist 组
        self.conv_h = nn.ModuleList([nn.Conv2d(1, 4, (i, dims)) for i in lengths])


        # vertical conv layer
        self.conv_v = nn.Conv2d(1, self.n_v, (L, 1))

        # fully-connected layer
        self.fc1_dim_v = self.n_v * dims
        self.fc1_dim_h = self.n_h * len(lengths)

    def forward(self, item_embs):
        out_v, out_h, out = 0,0,0

        if self.n_v: 
            out_v = self.conv_v(item_embs)
            out_v = out_v.view(-1, self.fc1_dim_v)  # prepare for fully connect
        out_hs = []
        
        if self.n_h:
            for conv in self.conv_h:
                conv_out = self.ac_conv(conv(item_embs).squeeze(3))
                pool_out = F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
                out_hs.append(pool_out)
            out_h = torch.cat(out_hs, 1)  # prepare for fully connect

        out = torch.cat([out_h, out_v], 1)
        return out

net = conv_demo()
res = net(item_embs)
print(res)
# 9+ 16 
print(res.size())

In [14]:
# all get up
class conv_demo(nn.Module):
    def __init__(self):
        super(conv_demo, self).__init__()
        # 先搬一搬 数据
        L = 4
        dims = 3
        self.n_h = 4
        self.n_v = 3
        self.drop_ratio = 0.2
        self.ac_conv = F.relu
        self.ac_fc = F.relu
        # 搬完了
        
        # horizontal conv layer 水平卷积  
        lengths = [i + 1 for i in range(L)]
        # n_h 为 4
        # modulelist 组
        self.conv_h = nn.ModuleList([nn.Conv2d(1, 4, (i, dims)) for i in lengths])


        # vertical conv layer
        self.conv_v = nn.Conv2d(1, self.n_v, (L, 1))

        # fully-connected layer
        self.fc1_dim_v = self.n_v * dims
        self.fc1_dim_h = self.n_h * len(lengths)
        fc1_dim_in = self.fc1_dim_h + self.fc1_dim_v
        
        # linear 1 
        self.fc1 = nn.Linear(fc1_dim_in, dims)
        # dropout
        self.dropout = nn.Dropout(self.drop_ratio)

        # linear 2 w2 b2  W2, b2 are encoded with nn.Embedding, as we don't need to compute scores for all items 这个代码人说的 
        self.W2 = nn.Embedding(10, 6)
        self.B2 = nn.Embedding(10, 1)

    def forward(self, item_embs, user_embs, list):
        out_v, out_h, out = 0,0,0

        if self.n_v: 
            out_v = self.conv_v(item_embs)
            out_v = out_v.view(-1, self.fc1_dim_v)  # prepare for fully connect
        out_hs = []
        
        if self.n_h:
            # 消边 
            for conv in self.conv_h:
                conv_out = self.ac_conv(conv(item_embs).squeeze(3))
                pool_out = F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
                out_hs.append(pool_out)
            out_h = torch.cat(out_hs, 1)  # prepare for fully connect

        out = torch.cat([out_h, out_v], 1)
        # apply dropout
        out = self.dropout(out)
        # 提前 return 会使 下面的代码失去颜色 
        # 就是没有 作用了

        # linear 1 
        z = self.ac_fc(self.fc1(out))
        x = torch.cat([z, user_embs],1)
        

        # 预测层
        w2 = self.W2(list)
        b2 = self.B2(list)
        # 和代码有较大出入 
        # 写文档！ 
        # 最后一行 写个跟狗一样
        res = torch.baddbmm(b2.unsqueeze(-1), w2.unsqueeze(1), x.unsqueeze(2)).squeeze()
        return res
net = conv_demo()
res = net(item_embs, user_embs, list)
print(res)
# 9+ 16 
print(res.size())



tensor([ 3.0754,  0.0949, -3.5818,  5.3011, -0.4682, -1.0110,  1.0467, -0.3563,
         1.5347,  2.5452], grad_fn=<SqueezeBackward0>)
torch.Size([10])


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH